# How to automatize the relaxation
In the first PbTe tutorial we saw how to prepare the SSCHA calculation from scratch and how to manipulate ensembles and the
minimization.
However, all the procedure could be a bit clumsy. In this section I will show how to automatize everything.
We will complete the PbTe simulation to get some usefull results.

The automatic relaxation is performed through the Relax module, that implements two classes, one for the static cell relaxation and the other for the variable cell relaxation

In [1]:
# Import all important stuff
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
# Import ASE to setup the automatic calculator
import ase
from ase.calculators.espresso import Espresso

# Lets import cellconstructor
import cellconstructor as CC
import cellconstructor.Phonons

# Import the sscha
import sscha, sscha.Ensemble, sscha.Relax, sscha.SchaMinimizer

To make everything clear, we will start again from the harmonic dynamical matrix for our structure:

    harmonic_dyn
    
We need to impose the sum rule and to force it to be positive definite before starting a SSCHA calculation

In [8]:
# Lets define the pseudopotentials
pseudos = {"Pb": "Pb.upf",
          "Te": "Te.upf"}

# Now we define the parameters for the espresso calculations
input_params = {"ecutwfc" : 60, # The plane-wave wave-function cutoff
               "ecutrho": 240, # The density wave-function cutoff,
               "conv_thr": 1e-6, # The convergence for the DFT self-consistency
               "pseudo_dir" : "pseudo_espresso", # The directory of the pseudo potentials
               "tprnfor" : True, # Print the forces
               "tstress" : True # Print the stress tensor
               }

k_spacing = 0.2 #A^-1 The minimum distance in the Brillouin zone sampling
k_offset = (0,0,0) # Offset for the K points grid

espresso_calc = Espresso(input_data = input_params, pseudopotentials = pseudos, kspacing = k_spacing,
                        koffset = k_offset)

In [9]:
dyn = CC.Phonons.Phonons("harmonic_dyn", nqirr = 3)

# Apply the symmetries and the sum rule
dyn.Symmetrize()

# Force it to be positive definite
dyn.ForcePositiveDefinite()

# Generate the ensemble and the minimizer objects
ensemble = sscha.Ensemble.Ensemble(dyn, 100, supercell = dyn.GetSupercell())
minimizer = sscha.SchaMinimizer.SSCHA_Minimizer(ensemble)

# We setup all the minimization parameters
minimizer.min_step_dyn = 0.02
minimizer.kong_liu_ratio = 0.3

You can automatize also the cluster calculation by setting up a cluster object 
like we did in the previous tutorials.
If you keep it as None (as done in the following cell) the calculation will be runned locally.

In [10]:
# We prepare the Automatic relaxation
relax = sscha.Relax.SSCHA(minimizer, ase_calculator = espresso_calc,
                         N_configs = 20,
                         max_pop = 6,
                         save_ensemble = True,
                         cluster = None)

Lets see the parameters:
    
    minimizer     : it is the SSCHA_Minimizer, containing the settings of each minimization.
    N_configs     : the number of configurations to generate at each run (we call them populations)
    max_pop       : The maximum number of populations after wich the calculation is stopped even if not converged.
    save_ensemble : If True, after each energy and force calculation, the ensemble will be saved.
    cluster       : The cluster object to be used for submitting the configurations.
    

If no cluster is provided (like in this case) the calculation is performed locally.
The save_ensemble keyword will dave the ensemble inside the directory specified in the running command
```python
relax.relax(ensemble_loc = "directory_of_the_ensemble")
```

The calculation can be runned just by calling
```python
relax.relax()
```
And the code will proceed with the automatic SSCHA minimization.
However, before doing so, we will setup a custom function.
These are usefull to manipulate a bit the minimization, printing or saving extra info
during the minimization.
In particular, we will setup a function to be called after each minimization step of each population, that
will save the current frequencies of the auxiliary dynamical matrix in an array, so that we will be able
to plot the whole frequency evolution after the minimization.

In [ ]:
# We reset the frequency array
all_frequencies = []

# We define a function that will be called by the code after each minimization step, 
# passing to us the minimizer at that point
# We will store the frequencies, to plot after the minimization their evolution.
def add_current_frequencies(minimizer):
    # Get the frequencies
    w, p = minimizer.dyn.DiagonalizeSupercell()
    
    all_frequencies.append(w)

# We add this function to the relax ojbect
relax.setup_custom_functions(custom_function_post = add_current_frequencies)

# Now we are ready to
# ***************** RUN *****************
relax.relax(ensemble_loc = "data_ensemble_autorelax")
# ***************************************

conf 0 / 20
conf 1 / 20
conf 2 / 20
conf 3 / 20
conf 4 / 20
conf 5 / 20
conf 6 / 20
conf 7 / 20
conf 8 / 20
conf 9 / 20
conf 10 / 20
conf 11 / 20
conf 12 / 20
conf 13 / 20
conf 14 / 20
conf 15 / 20
conf 16 / 20
conf 17 / 20
conf 18 / 20
conf 19 / 20


/home/pione/anaconda2/lib/python2.7/site-packages/sscha/Ensemble.py:839: ComplexWarning: Casting complex values to real discards the imaginary part
  self.sscha_energies[i], self.sscha_forces[i, :,:] = new_super_dyn.get_energy_forces(self.structures[i], real_space_fc = new_super_dyn.dynmats[0], displacement = self.u_disps[i, :])


Time elapsed to prepare the rho update: 0.215842962265 s
(of which to update sscha energies and forces: 0.0019 s)
(of which computing the Upsilon matrix: 0.1113 s)
Time elapsed to update weights the sscha energies, forces and displacements: 0.114411115646 s
(of which to update the weights): 0.00128507614136 s
 [GRADIENT] Time to prepare the gradient calculation: 0.0513579845428 s
 [GRADIENT] Time to call the fortran code: 0.0181601047516 s
 [GRADIENT] Time to get back in fourier space: 0.0355529785156 s

 # ---------------- NEW MINIMIZATION STEP --------------------
Step ka =  1
 [GRADIENT] Time to prepare the gradient calculation: 0.0340180397034 s
 [GRADIENT] Time to call the fortran code: 0.0133872032166 s
 [GRADIENT] Time to get back in fourier space: 0.0329728126526 s
Time elapsed to symmetrize the gradient: 0.0210101604462 s
Time elapsed to compute the structure gradient: 0.00241613388062 s
Time elapsed to prepare the rho update: 0.167854070663 s
(of which to update sscha energie